### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5457,2023-11-01,Brasil Nbb,20:00,Caxias do Sul,Vasco da Gama,2.00,1.71,145.5,1.83,1.83,-4.5,1.83,1.83
5458,2023-11-01,Eua Nba,21:00,Houston Rockets,Charlotte Hornets,1.77,2.10,222.5,1.91,1.91,-2.5,1.91,1.91
5459,2023-11-01,Eua Nba,22:00,Utah Jazz,Memphis Grizzlies,1.69,2.25,228.5,1.91,1.91,-2.5,1.91,1.91
5460,2023-11-01,Europa Eurocopa,14:00,Besiktas,Venezia,1.43,2.90,155.5,1.91,1.91,-5.5,1.91,1.91
5461,2023-11-01,Europa Eurocopa,14:00,Prometey,Paris,2.50,1.59,172.5,1.91,1.91,4.5,1.91,1.91
5462,2023-11-01,Europa Eurocopa,15:00,Aris,Cluj-Napoca,1.80,2.05,154.5,1.91,1.91,-1.5,1.91,1.91
5463,2023-11-01,Europa Eurocopa,15:30,JL Bourg,Buducnost,1.59,2.50,160.5,1.95,1.87,-3.5,1.91,1.91
5464,2023-11-01,Europa Eurocopa,15:30,Ulm,Slask Wroclaw,1.07,10.00,166.5,1.91,1.91,-14.5,1.91,1.91
5465,2023-11-01,Europa Eurocopa,17:00,Gran Canaria,Lietkabelis,1.08,9.00,162.5,1.91,1.91,-14.5,1.95,1.87
5466,2023-11-01,Austrália Liga Feminina Wnbl,05:00,Adelaide Lightning F,Melbourne Boomers F,4.33,1.20,161.5,1.87,1.87,8.5,1.87,1.87


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
1,21:00,Eua Nba,Houston Rockets,Charlotte Hornets,222.5,1.91,0.7954,Over
2,22:00,Eua Nba,Utah Jazz,Memphis Grizzlies,228.5,1.91,0.7778,Over
3,14:00,Europa Eurocopa,Besiktas,Venezia,155.5,1.91,0.8445,Over
4,14:00,Europa Eurocopa,Prometey,Paris,172.5,1.91,0.8092,Over
5,15:00,Europa Eurocopa,Aris,Cluj-Napoca,154.5,1.91,0.8456,Over
6,15:30,Europa Eurocopa,JL Bourg,Buducnost,160.5,1.95,0.7688,Over
7,15:30,Europa Eurocopa,Ulm,Slask Wroclaw,166.5,1.91,0.8534,Over
8,17:00,Europa Eurocopa,Gran Canaria,Lietkabelis,162.5,1.91,0.7561,Over


### AH Zebra V1 (LR)

In [9]:
# Duplicando o dataset
df_ah_pos_v1 = jogos.copy()

# AH
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v1['Zebra'] = df_ah_pos_v1['Zebra'].astype('category')
# Odds
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v1['HA_Odds_H']
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v1['HA_Odds_A']

df_ah_pos_v1.reset_index(drop=True, inplace=True)
df_ah_pos_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v1.empty:
    mdl_ah_pos_v1 = load_model('ML/ah_pos/v1/v1_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v1, df_ah_pos_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] > 1.85))]
    prev = prev[((prev['Over_Line'] >= 200) & (prev['Over_Line'] < 212)) | ((prev['Over_Line'] >= 215) & (prev['Over_Line'] < 218))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V1 (LR) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V1 (LR) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
11,08:30,China Cba,Guangdong,Tianjin,-13.5,1.87,0.5492,AH Zebra


### AH Zebra V2 (ADA)

In [10]:
# Duplicando o dataset
df_ah_pos_v2 = jogos.copy()

# AH
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v2['Zebra'] = df_ah_pos_v2['Zebra'].astype('category')
# Odds
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v2['HA_Odds_H']
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v2['HA_Odds_A']

df_ah_pos_v2.reset_index(drop=True, inplace=True)
df_ah_pos_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v2.empty:
    mdl_ah_pos_v2 = load_model('ML/ah_pos/v2/v2_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v2, df_ah_pos_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V2 (ADA) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V2 (ADA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
10,08:30,China Cba,Fujian,Jiangsu Dragons,-8.5,1.87,0.5004,AH Zebra
12,08:30,China Cba,Guangzhou,Beijing Royal Fighters,-7.5,1.87,0.5045,AH Zebra


### AH Zebra V3 (ADA - SEED 8941)

In [11]:
# Duplicando o dataset
df_ah_pos_v3 = jogos.copy()

# AH
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v3['Zebra'] = df_ah_pos_v3['Zebra'].astype('category')
# Odds
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v3['HA_Odds_H']
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v3['HA_Odds_A']

df_ah_pos_v3.reset_index(drop=True, inplace=True)
df_ah_pos_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v3.empty:
    mdl_ah_pos_v3 = load_model('ML/ah_pos/v3/v3_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v3, df_ah_pos_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
10,08:30,China Cba,Fujian,Jiangsu Dragons,-8.5,1.87,0.5005,AH Zebra


### AH Zebra V4 (ADA - SEED 4280)

In [12]:
# Duplicando o dataset
df_ah_pos_v4 = jogos.copy()

# AH
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v4['Zebra'] = df_ah_pos_v4['Zebra'].astype('category')
# Odds
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v4['HA_Odds_H']
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v4['HA_Odds_A']

df_ah_pos_v4.reset_index(drop=True, inplace=True)
df_ah_pos_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v4.empty:
    mdl_ah_pos_v4 = load_model('ML/ah_pos/v3/v4_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v4, df_ah_pos_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
10,08:30,China Cba,Fujian,Jiangsu Dragons,-8.5,1.87,0.5015,AH Zebra
